In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

## Hämta comedrivern: https://googlechromelabs.github.io/chrome-for-testing/

In [2]:
import os

# Dynamically increase the data rate limit
os.environ["SERVERAPP_IOPUB_DATA_RATE_LIMIT"] = "1000000000"
os.environ["SERVERAPP_RATE_LIMIT_WINDOW"] = "25"

In [3]:
service = Service("chromedriver-win64/chromedriver.exe")
driver = webdriver.Chrome(service=service)

##Huvudstadsreg + periferier + höghus och rad/parhus
url = 'https://asunnot.oikotie.fi/myytavat-asunnot?pagination=1&locations=%5B%5B64,6,%22Helsinki%22%5D,%5B39,6,%22Espoo%22%5D,%5B65,6,%22Vantaa%22%5D,%5B147,6,%22Kirkkonummi%22%5D,%5B359,6,%22Sipoo%22%5D%5D&cardType=100&buildingType%5B%5D=1&buildingType%5B%5D=256&buildingType%5B%5D=2&buildingType%5B%5D=64'

#Hangö Lovisa
#url = 'https://asunnot.oikotie.fi/myytavat-asunnot?pagination=1&locations=%5B%5B220,6,%22Loviisa%22%5D,%5B55,6,%22Hanko%22%5D%5D&cardType=100'

driver.get(url)

In [4]:
products = driver.find_elements(by="xpath", value='//div[contains(@class, "ot-card-v2__info-container padded padded--v20-h2")]')

In [5]:
Plats = []
pris_storlke = []
Ovrigt = []

In [6]:

def click_next_page_button(driver):
    try:
        # Wait until the button with analytics-click-label="right" is present and clickable
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '[analytics-click-label="right"]'))
        )
        # Click the button
        button.click()
        print("Button clicked successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [7]:

def scrape_page(driver):
    try:
        # Extract product information
        products = driver.find_elements(By.XPATH, '//div[contains(@class, "ot-card-v2__info-container padded padded--v20-h2")]')

        for product in products:
            try:
                Plats.append(product.find_element(By.XPATH, './/div[contains(@class, "card-v2-text-container__text card-v2-text-container__text--bold")]').text)
            except:
                Plats.append(None)  # Append None if the element is not found

            try:
                pris_storlke.append(product.find_element(By.XPATH, './/div[contains(@class, "card-v2-text-container__group card-v2-text-container__group--boxed")]').text)
            except:
                pris_storlke.append(None)

            try:
                Ovrigt.append(product.find_element(By.XPATH, './/div[contains(@class, "card-v2-text-container__group card-v2-text-container__group--separated")]').text)
            except:
                Ovrigt.append(None)

        return Plats, pris_storlke, Ovrigt
    except Exception as e:
        print(f"An error occurred while scraping the page: {e}")
        return [], [], []


In [8]:
def scrape_all_pages(driver):
    all_Plats = []
    all_pris_storlke = []
    all_Ovrigt = []

    while True:
        # Scrape the current page
        Plats, pris_storlke, Ovrigt = scrape_page(driver)
        all_Plats.extend(Plats)
        all_pris_storlke.extend(pris_storlke)
        all_Ovrigt.extend(Ovrigt)

        try:
            # Click the "Next Page" button and wait for the page to load
            click_next_page_button(driver)
            
            # Wait for the page content to be refreshed
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "ot-card-v2__info-container padded padded--v20-h2")]'))
            )
            print("Moved to the next page.")
        except Exception as e:
            print(f"Failed to move to the next page or load next page content: {e}")
            break

    # Process the collected data
    print(f"Plats: {all_Plats}")
    print(f"Pris Storleke: {all_pris_storlke}")
    print(f"Ovrigt: {all_Ovrigt}")

In [9]:
scrape_all_pages(driver)

Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [10]:
Plats

['Karakalliontie 14 C, Karakallio, Espoo',
 'Korkeavuorenkatu 5 A 9, Ullanlinna, Helsinki',
 'Unioninkatu 8 A, Kaartinkaupunki, Helsinki',
 'Tehtaankatu 42 A, Eira, Helsinki',
 'Norppatie 7 E, Hyljelahti, Espoo',
 'Alakiventie 3 F, Myllypuro, Helsinki',
 'Terhotie 1 E 6, Pähkinärinne, Vantaa',
 'Nihtisillankuja 2, Kilo, Espoo',
 'Pohjoiskaari 12 B, Lauttasaari, Helsinki',
 'Läntinen Brahenkatu 4b A, Alppiharju, Helsinki',
 'Käskynhaltijantie 40 C2, Oulunkylä, Helsinki',
 'Bertha Pauligin katu 2 a, Aurinkolahti, Helsinki',
 'Kaupintie 4 F 117, Haaga, Helsinki',
 'Pääskylänrinne 4 B, Sörnäinen, Helsinki',
 'Iivisniementie 2 F, Iivisniemi, Espoo',
 'Vasamatie 2, Kilo, Espoo',
 'Kaskelankuja 12d A 1, Kaskela, Vantaa',
 'Malminkatu 22 B, Kamppi, Helsinki',
 'Strömsintie 21, Marjaniemi, Helsinki',
 'Klaukkalantie 32 B, Itä-Pakila, Helsinki',
 'Punavuorenkatu 20 C, Punavuori, Helsinki',
 'Munkkiniemen puistotie 20, Munkkiniemi, Helsinki',
 'Hankainpolku 5, Vuosaari, Helsinki',
 'Saloviuksenpi

In [11]:
df_plats = pd.DataFrame([x.split(', ') for x in Plats], columns=['Street Address', 'Area', 'City', "Random"])
#df_plats = pd.DataFrame([x.split(', ') for x in Plats], columns=['Street Address', 'Area', 'City', "Random"])


In [12]:
df_plats

,Street Address,Area,City,Random
0,Karakalliontie 14 C,Karakallio,Espoo,None
1,Korkeavuorenkatu 5 A 9,Ullanlinna,Helsinki,None
2,Unioninkatu 8 A,Kaartinkaupunki,Helsinki,None
3,Tehtaankatu 42 A,Eira,Helsinki,None
4,Norppatie 7 E,Hyljelahti,Espoo,None
...,...,...,...,...
13253,Ojalehdonkuja 2 A 38,Asola,Vantaa,None
13254,Ojalehdonkuja 2 A 29,Asola,Vantaa,None
13255,Steniuksentie 21,Haaga,Helsinki,None
13256,Ristolantie 5,Haaga,Helsinki,None


In [13]:
# Splitting the list based on new lines
split_data = [x.split('\n') for x in pris_storlke]

# Determine the maximum number of parts
max_parts = max(len(item) for item in split_data)

# Create a DataFrame with columns for each part, filling missing values with NaN
columns = ['Price', 'Size', 'Maintenance', 'Price_per_sqm', 'Floors', 'Other']
#columns = ['Price', 'Size']

df_pris_storlk = pd.DataFrame([item + [np.nan] * (max_parts - len(item)) for item in split_data], columns=columns)


In [14]:
# Initialize lists to hold the split components
types = []
years = []
details = []

# Loop through the Ovrigt list and split each item
for item in Ovrigt:
    # Split on the first newline character
    parts = item.split('\n', 1)
    
    # Further split the first part to get the type and year
    type_year = parts[0].split(', ')
    
    # Assign values to type, year, and details
    type_val = type_year[0] if len(type_year) > 0 else None
    year_val = type_year[1] if len(type_year) > 1 else None
    detail_val = parts[1] if len(parts) > 1 else None
    
    # Append the values to the respective lists
    types.append(type_val)
    years.append(year_val)
    details.append(detail_val)

# Creating a DataFrame with columns for 'Type', 'Year', and 'Details'
df_ovrigt = pd.DataFrame({
    'Type': types,
    'Year': years,
    'Details': details
})



In [15]:
combined_df = pd.concat([df_plats, df_pris_storlk, df_ovrigt], axis=1)


In [16]:
combined_df

,Street Address,Area,City,Random,Price,Size,Maintenance,Price_per_sqm,Floors,Other,Type,Year,Details
0,Karakalliontie 14 C,Karakallio,Espoo,None,178 000 €,72 m²,NaN,NaN,NaN,NaN,Kerrostalo,1968,"3h, k, kph, erillis-WC, et, lasitettu parveke"
1,Korkeavuorenkatu 5 A 9,Ullanlinna,Helsinki,None,998 000 €,114 m²,NaN,NaN,NaN,NaN,Kerrostalo,1928,3mh+oloh+keittiö ja ruokatila+sauna+kph+wc
2,Unioninkatu 8 A,Kaartinkaupunki,Helsinki,None,426 000 €,"47,5 m²",NaN,NaN,NaN,NaN,Kerrostalo,1962,2 h+kt (yhtiöjärjestyksen mukaan)
3,Tehtaankatu 42 A,Eira,Helsinki,None,358 000 €,42 m²,NaN,NaN,NaN,NaN,Kerrostalo,2000,2h+kk+s (yhtiöjärjestyksen mukaan 2h+kk+s)
4,Norppatie 7 E,Hyljelahti,Espoo,None,259 000 €,74 m²,NaN,NaN,NaN,NaN,Luhtitalo,1989,"3h, k, kph, wc, s, vh, p"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13253,Ojalehdonkuja 2 A 38,Asola,Vantaa,None,303 500 €,76 m²,NaN,NaN,NaN,NaN,Kerrostalo,2021,4h+kt
13254,Ojalehdonkuja 2 A 29,Asola,Vantaa,None,245 800 €,"59,5 m²",NaN,NaN,NaN,NaN,Kerrostalo,2021,3h+kt
13255,Steniuksentie 21,Haaga,Helsinki,None,215 000 €,28 m²,NaN,NaN,NaN,NaN,Kerrostalo,1957,1h + kk+kph+vh
13256,Ristolantie 5,Haaga,Helsinki,None,275 000 €,"28,5 m²",NaN,NaN,NaN,NaN,Kerrostalo,1955,1h+kk+ransk. parveke


In [17]:
combined_df.to_excel('Utdata/All_data_Huvudstadsreg_o_periferi.19.01.2025.xlsx', index=False)
#combined_df.to_excel('Utdata/Lovisa_hangö/Lovisa_hangö_all_boende.13.10.2024.xlsx', index=False)


In [ ]:
## ot-card-v2__info-container padded padded--v20-h20    Lådan
#//div[contains(@class, "ot-card-v2__info-container padded padded--v20-h2")] Lådan
#card-v2-text-container__column--desktop-wide,  priset 
# //div[contains(@class, "card-v2-text-container__column--desktop-wide")]  priset



#//div[contains(@class, "card-v2-text-container__text card-v2-text-container__text--bold ")] Plats/ID


#"card-v2-text-container__column" Km2
# //div[contains(@class, "card-v2-text-container__column")]




